# ball-tree-distance

In [ ]:
import glob

import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits
from sklearn.neighbors import BallTree

######################################
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v6/'
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/dr7_all.fits'


def distance(X1, X2, max_r):

    X1 = X1 * (np.pi / 180.)
    X2 = X2 * (np.pi / 180.)
    max_distance = max_r * (np.pi / 180.)

    # Convert 2D RA/DEC to 3D cartesian coordinates
    Y1 = np.transpose(
        np.vstack([
            np.cos(X1[0]) * np.cos(X1[1]),
            np.sin(X1[0]) * np.cos(X1[1]),
            np.sin(X1[1])
        ]))
    Y2 = np.transpose(
        np.vstack([
            np.cos(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 1])
        ]))

    # law of cosines to compute 3D distance
    max_y = np.sqrt(2 - 2 * np.cos(max_distance))

    tree = BallTree(Y2)
    num = tree.query_radius(Y1, r=max_distance, count_only=True)
    ind, dist = tree.query_radius(Y1,
                                  r=max_distance,
                                  count_only=False,
                                  return_distance=True)
    return num, ind, dist


def r2arcsec(r):
    return r * 180 / np.pi


def max_dis_deg(z):
    '''
    150kpc ~= degree
    '''
    dis = cosmo.comoving_distance(z)  # Mpc
    r = 0.15 / dis.value
    max_r = r2arcsec(r)  # degree
    return max_r


def cross():
    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        data = f_drp[1].data
        manga = np.empty((len(data), 2), dtype=np.float64)
        manga[:, 0] = data['objra']
        manga[:, 1] = data['objdec']
        manga_z = data['z']
        plateifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0, len(manga)):
            pifu = plateifu[i]
            z = manga_z[i]
            max_r = max_dis_deg(z)
            num, ind, dist = distance(manga[i], dr7, max_r)
            array = []
            names = [
                'plateifu', 'manga_ra', 'manga_dec', 'manga_z', 'dr7_index',
                'dr7_ra', 'dr7_dec', 'dr7_z', 'distance', 'max_r'
            ]
            dtype=['U11','f8','f8','f8','i4','f8','f8','i4','f8','f8']
            if num[0] > 0:
                for j in range(0, num[0]):
                    index = ind[0][j]
                    array.append([
                        pifu, manga[i][0], manga[i][1], manga_z[i],index,
                        dr7[index][0], dr7[index][1],dr7_z[index],dist[0][j],max_r
                    ])
#                 print(array)
                array=np.array(array)
                t=Table(array,names=names)
#                 print(t)
                t.write(dir_res+pifu+'.fits',format='fits')
            else:
                with open(dir_res+'no_match.txt','a+') as f:
                    print(pifu,file=f)
if __name__ == '__main__':
    cross()

In [24]:
import numpy as np
import os
from sklearn.neighbors import BallTree
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

####################################################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/dr7_all.fits'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v5/cmatch_10_not_enough.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v5/'
####################################################################################################


def match(X1, X2, max_distance):
    X1 = X1 * (np.pi / 180.)
    X2 = X2 * (np.pi / 180.)
    max_distance = max_distance * (np.pi / 180.)

    # Convert 2D RA/DEC to 3D cartesian coordinates
    Y1 = np.transpose(
        np.vstack([
            np.cos(X1[0]) * np.cos(X1[1]),
            np.sin(X1[0]) * np.cos(X1[1]),
            np.sin(X1[1])
        ]))
    Y2 = np.transpose(
        np.vstack([
            np.cos(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 1])
        ]))

    max_r = np.sqrt(2 - 2 * np.cos(max_distance))

    tree = BallTree(Y2)
    num = tree.query_radius(Y1, r=max_r, count_only=True)
    ind = tree.query_radius(Y1, r=max_r)
    return num, ind


def r2arcsec(r):
    return r * 180 / np.pi


def max_dis_deg(z):
    '''
    150kpc ~= degree
    '''
    dis = cosmo.comoving_distance(z)  # Mpc
    r = 0.15 / dis.value
    max_r = r2arcsec(r)  # degree
    return max_r


def cmatch():
    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        data = f_drp[1].data
        manga = np.empty((len(data), 2), dtype=np.float64)
        manga[:, 0] = data['objra']
        manga[:, 1] = data['objdec']
        manga_z = data['z']
        plateifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0,len(manga)):
            
            z = manga_z[i]
            pifu = plateifu[i]
            if os.path.exists(dir_res+pifu+'.fits'):
                continue
            else:
                max_r = max_dis_deg(z)
                num,ind = match(manga[i],dr7,max_r)
                array=[]
                names=['ra','dex','z','index']
                if num[0]>0:
                    for j in range(0,num[0]):

                        index=ind[0][j]
                        array.append([dr7[index,0],dr7[index,1],dr7_z[index],index])

                    array=np.array(array)
                    t=Table(array,names=names,dtype=['f8','f8','f8','i4'])
                    t.write(dir_res+pifu+'.fits',format='fits')
                else:
                    with open(dir_res+'no_match.txt','a+') as f:
                        print(plateifu[i],file=f)
            
if __name__=='__main__':
    cmatch()

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

## test_distance

In [10]:

import glob
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits

dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v6/'
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/dr7_all.fits'

file=glob.glob(dir_res+'*fits')
with fits.open('/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits') as f_drp:
    data_drp=f_drp[1].data
    plateifu_drp=data_drp['plateifu']
    ra_drp=data_drp['objra']
    dec_drp=data_drp['objdec']
    z=data_drp['z']
    for filename in file:
        plateifu=filename[len(dir_res):-5]
#         print(plateifu)
        index=np.where(plateifu_drp==plateifu)[0]
        ra1=ra_drp[index]
        dec1=dec_drp[index]
        z1=z[index]
        dis=Distance(z=z1).value
        theta=0.15/(dis)
        with fits.open(filename) as f:
            data=f[1].data
            ra=data['dr7_ra']
            dec=data['dr7_dec']
            
            # str to float
            ra=[float(n) for n in ra]
            dec=[float(m) for m in dec]
            if len(ra)==0:
                print('null',plateifu)
            else:
#                 print(ra,dec)
                manga=SkyCoord(ra=ra1*u.deg,dec=dec1*u.deg)
                catalog=SkyCoord(ra=ra*u.deg,dec=dec*u.deg)
                dis=manga.separation(catalog).radian
                dis_max=np.max(dis)

                if dis_max<=theta:
                    continue
                else:
                    print('wrong',plateifu)


wrong 10001-12704
wrong 10001-3702
wrong 10001-6102
wrong 10001-6103
wrong 10213-3701
wrong 10214-3702
wrong 10215-3701
wrong 10216-12702
wrong 10216-12705
wrong 10216-1901
wrong 10216-3702
wrong 10217-12703
wrong 10217-12704
wrong 10217-12705
wrong 10217-1901
wrong 10218-3702
wrong 10219-1902
wrong 10219-3702
wrong 10219-3704
wrong 10220-12701
wrong 10221-12703
wrong 10221-3703
wrong 10221-3704
wrong 10221-6103
wrong 10492-12705
wrong 10492-1902
wrong 10492-3703
wrong 10493-1901
wrong 10493-1902
wrong 10493-3701
wrong 10493-3704
wrong 10493-9102
wrong 10494-12705
wrong 10494-1901
wrong 10494-3701
wrong 10494-6102
wrong 10494-6104
wrong 10496-6102
wrong 10497-6103
wrong 10497-6104
wrong 10498-12701
wrong 10498-12702
wrong 10498-12704
wrong 10498-1901
wrong 10498-3703
wrong 10498-6101
wrong 10499-12701
wrong 10499-1901
wrong 10499-1902
wrong 10499-3701
wrong 10499-6101
wrong 10503-12701
wrong 10503-12705
wrong 10503-6102
wrong 10503-9101
wrong 10503-9102
wrong 10504-12701
wrong 10504-12

wrong 8481-3702
wrong 8482-3701
wrong 8483-12704
wrong 8483-1901
wrong 8483-6102
wrong 8483-9102
wrong 8484-12703
wrong 8486-12704
wrong 8486-1901
wrong 8486-9101
wrong 8547-9101
wrong 8549-1902
wrong 8549-3702
wrong 8550-12702
wrong 8550-3703
wrong 8550-6103
wrong 8551-1901
wrong 8551-3701
wrong 8551-3704
wrong 8552-12702
wrong 8552-1901
wrong 8552-3701
wrong 8553-12702
wrong 8553-3701
wrong 8554-12704
wrong 8554-3701
wrong 8555-3701
wrong 8555-9102
wrong 8566-12703
wrong 8566-12705
wrong 8566-3702
wrong 8566-6103
wrong 8566-6104
wrong 8567-12703
wrong 8568-12702
wrong 8568-12704
wrong 8568-12705
wrong 8568-3701
wrong 8568-6102
wrong 8588-12701
wrong 8588-12702
wrong 8588-12704
wrong 8588-3701
wrong 8591-12704
wrong 8591-1902
wrong 8591-3701
wrong 8591-6102
wrong 8591-6103
wrong 8592-12705
wrong 8592-1902
wrong 8592-3701
wrong 8592-3704
wrong 8593-12701
wrong 8593-1902
wrong 8593-3703
wrong 8593-6103
wrong 8597-9101
wrong 8600-1901
wrong 8600-3703
wrong 8600-3704
wrong 8601-12701
wron

wrong 9194-1902
wrong 9194-3701
wrong 9194-3702
wrong 9194-6103
wrong 9194-9101
wrong 9195-12701
wrong 9195-6102
wrong 9195-9102
wrong 9196-1902
wrong 9196-3701
wrong 9485-12701
wrong 9485-12702
wrong 9485-12705
wrong 9486-12702
wrong 9486-12704
wrong 9486-3701
wrong 9486-3702
wrong 9486-6103
wrong 9487-12704
wrong 9487-1902
wrong 9487-9101
wrong 9488-12701
wrong 9488-12702
wrong 9488-12703
wrong 9488-12704
wrong 9488-12705
wrong 9488-1901
wrong 9488-6101
wrong 9488-6104
wrong 9489-12702
wrong 9489-12704
wrong 9489-3702
wrong 9489-6101
wrong 9489-9102
wrong 9490-12704
wrong 9490-3702
wrong 9490-3704
wrong 9490-6103
wrong 9491-12702
wrong 9491-1901
wrong 9491-3704
wrong 9492-1902
wrong 9492-3704
wrong 9493-12702
wrong 9494-12701
wrong 9494-12703
wrong 9494-12705
wrong 9494-3704
wrong 9494-6102
wrong 9494-6103
wrong 9496-1901
wrong 9497-12704
wrong 9497-9101
wrong 9498-6101
wrong 9499-12703
wrong 9501-9102
wrong 9502-12704
wrong 9502-6101
wrong 9503-12701
wrong 9503-3704
wrong 9503-6102


In [28]:
from astropy.coordinates import Distance

dis=Distance(z=0.03)
dis.value

135.79570049977337

# velocity

In [14]:
import glob
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits
from astropy.table import Table

dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v6/'
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/dr7_all.fits'
dir_res1=dir_res+'150_1000/'

file=glob.glob(dir_res+'/*fits')
for filename in file:
    f=Table.read(filename)
    dr7_z=f['dr7_z']
    manga_z=f['manga_z']
    delta_z=[abs(float(n)-float(m)) for n,m in zip(dr7_z,manga_z)]
    delta_z=np.array(delta_z)
    index=np.where(delta_z<1/300)[0]
    pifu=filename[len(dir_res):-5]
    if len(index)>0:
#     print(index)
#     print(f[index])       
        f[index].write(dir_res1+pifu+'.fits',format='fits')
    else:
        with open(dir_res1+'no_match.txt','a+') as f1:
            print(pifu,file=f1)
        